In [1]:
import pandas as pd
import calendar

#read the data
df = pd.read_excel('covid-19-state-level-data.xlsx', engine='openpyxl')
df['date'] = pd.to_datetime(df['date'])

#finding trend for the current month whenever prompted the current month trend will be sent to the group
from datetime import datetime, date

# Get the current date
current_date = date.today()
year = 2020

# Get the start and end dates of the current month
start__date = datetime(year, current_date.month, 1).strftime('%m/%d/%Y')
_, last_day = calendar.monthrange(year, current_date.month)
end__date = datetime(year, current_date.month, last_day).strftime('%m/%d/%Y')


start_date = pd.to_datetime(start__date)
end_date = pd.to_datetime(end__date)

sliced_data = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
# Assuming the sliced dataset is stored in the variable 'sliced_data'
total_value_per_name = sliced_data.groupby('state')['deaths'].sum()

total_sum = total_value_per_name.sum()
top_3_values = total_value_per_name.nlargest(3)

current_month = datetime.now().strftime('%B')
output_list = []

output_list.append(f"Top 3 states with the highest number of COVID deaths for the month of {current_month}\n")
for name, value in top_3_values.items():
    percentage = (value / total_sum) * 100
    output_list.append(f"State: {name}, no. of deaths: {value}, % of total US deaths: {percentage:.2f}%")

output_variable = '\n'.join(output_list)

print(output_variable)
 

Top 3 states with the highest number of COVID deaths for the month of May

State: New York, no. of deaths: 854088, % of total US deaths: 31.69%
State: New Jersey, no. of deaths: 308935, % of total US deaths: 11.46%
State: Massachusetts, no. of deaths: 170827, % of total US deaths: 6.34%


In [2]:
import plotly.express as px

# Create a DataFrame from the top 3 values
df_top_3 = pd.DataFrame({'States': top_3_values.index, 'Value': top_3_values.values})

# Calculate the percentage for each value
df_top_3['Percentage'] = (df_top_3['Value'] / total_sum) * 100

# Create an interactive bar chart using Plotly
fig = px.bar(df_top_3, x='States', y='Percentage', title='percentage of total US deaths')
fig.show()

In [3]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage

# Assuming fig is the figure object representing the Plotly chart

# Email configuration
sender_email = 'mdhumayun53@gmail.com'
sender_password = 'gldvtrapqxxycnce'
receiver_emails = ['finkfloyddsotm73@gmail.com', 'mdhumayun53@gmail.com']  # List of receiver email addresses

# Create the email message
msg = MIMEMultipart()
msg['From'] = sender_email
msg['To'] = ', '.join(receiver_emails)
msg['Subject'] = 'Monthly Summary'

# Attach the top 3 values as text
output_variable_text = str(output_variable)
text_part = MIMEText(output_variable_text, 'plain')
msg.attach(text_part)

# Save the chart as an image and attach it to the email
fig.write_image('newplot.png')
with open('newplot.png', 'rb') as image_file:
    image_data = image_file.read()
image_part = MIMEImage(image_data)
image_part.add_header('Content-Disposition', 'attachment', filename='newplot.png')
msg.attach(image_part)

# Send the email
with smtplib.SMTP('smtp.gmail.com', 587) as server:
    server.starttls()
    server.login(sender_email, sender_password)
    server.send_message(msg)
